In [1]:

from pathlib import Path
import sys
import os

# path_root = Path(__file__).parents[1]
# sys.path.append(str(path_root))
# os.chdir('..')
%cd ..
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from os.path import join as jn
import yaml

import torch
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import json
from natsort import natsorted


from video_module import Stack_dataset, \
    fit_epoch, eval_epoch, predict, eval_dataset

/home/amir/projects/frame_stack


In [2]:
with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)
with open('pathes.yaml') as conf_file:
    path_config = yaml.safe_load(conf_file)

if not os.path.exists(path_config['reports_path']):
    os.makedirs(path_config['reports_path'])

In [3]:
torch.manual_seed(config['random_seed'])
np.random.seed(config['random_seed'])


In [7]:

input_path = path_config['s_video_path']
output_path = path_config['p_video_path']

tr = config['video_train']
frames_number, frames_interval = tr["frames_number"], tr["frames_interval"]

test_dataset = Stack_dataset(jn(output_path, 'test'), jn(input_path, 'test'), frames_number,
                             frames_interval)
train_dataset = Stack_dataset(jn(output_path, 'train'), jn(input_path, 'train'), frames_number,
                              frames_interval)



Dataset loading:   0%|                                                                                        …

  0%|                                                                                                         …

Dataset loading:   0%|                                                                                        …

  0%|                                                                                                         …

In [4]:

if not torch.cuda.is_available():
    print('CUDA is NOT available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


CUDA is available!  Training on GPU ...


In [8]:
with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)
    
signal_shape, pressure_shape = (x.shape for x in train_dataset[0])
print('input chain shape: ', signal_shape, '\noutput chain shape: ',
      pressure_shape)

model_name = tr['model_name']
import models_src

model_class = eval(f"models_src.{model_name}")

args = []
if model_name.startswith("Param"):
    layers = tr['layers']
    args.append(layers)
args.append(frames_number)
args.append(frames_interval)
model = model_class(pressure_shape[-2:], signal_shape[-2:], *args)
model = model.to(device)

# print(model)
optim = torch.optim.Adam(model.parameters(), lr=tr['learning_rate'])
loss_fn = torch.nn.MSELoss()


input chain shape:  (1, 4, 64) 
output chain shape:  (64, 64)


In [9]:
checkp_dir = 'logdir/checkpoints/'
if not os.path.exists(checkp_dir):
    os.makedirs(checkp_dir)

def iter_train(train_dataset, test_dataset, model, epochs, optimizer,
               criterion):
    for epoch in range(epochs):
        train_loss = fit_epoch(model, train_dataset, criterion, optimizer,
                               tr['batch_size'], device)
        test_loss = eval_epoch(model, test_dataset, criterion,
                               config['test_batch_size'], device)
        # print("loss", f"{train_loss:.3f}")
        # pbar.set_postfix(train_loss=train_loss, test_loss=test_loss)
        # full_train_loss = eval_dataset(model, train_dataset, criterion, config['test_batch_size'], device)
        # full_test_loss = eval_dataset(model, test_dataset, criterion, config['test_batch_size'], device)
        yield (train_loss, test_loss)
        
from video_module import Dynamic_video_dataset, predict, visual_chains
import h5py

def make_visualization(save_path):
    from video_module import Dynamic_video_dataset, predict, visual_chains

    v_model = model

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    def visual_dataset(pres_file, signal_file, step, max_items, begin=0, end=None, duration=30):
        prev_id = ''
        total=min(len(signal_file.keys()) // step, max_items)
        for key in tqdm(natsorted(signal_file.keys())[:total*step:step], total=total, leave=False):
            
            pressure = pres_file[key][begin:end]
            signal = signal_file[key][begin:end]
            
            id = key[:key.rfind('_')]
            if id != prev_id:
                print(f"\n#### id = {id}", file=file)
                prev_id = id
            prediction = predict(v_model,
                                signal,
                                device)     
            pressure = pressure[-prediction.shape[0]:]
            visual_chains([pressure, prediction],
                        jn(save_path, key), duration=duration)
            print(f"<img src={key+'.gif'} width=400>",
                file=file)

    file = open(jn(save_path, "view.md"), 'w')
    print("# Visualization", file=file)

    for data_path in config['visual']:
        with h5py.File(jn(output_path, data_path + '.hdf5')) as pres_file, \
            h5py.File(jn(input_path, data_path + '.hdf5')) as signal_file:
            print(f"\n# {data_path}", file=file)
            visual_dataset(pres_file, signal_file, **config['visual'][data_path])
    
    


In [14]:
history = []

In [10]:
epochs = 30 # config['video_train']['epochs']

In [ ]:
with tqdm(total=epochs,
          position=0,
          unit='epoch',
          desc="Learning",
          dynamic_ncols=True) as pbar:
    
    for h in iter_train(train_dataset, test_dataset, model, epochs, optim,
                        loss_fn):
        history.append(h)
        train_loss, test_loss = h
        # print(f"Epoch {i+1}/{total_epochs}",
        #       f"train loss: {full_train_loss:.5f}, test_loss: {full_test_loss:.5f}")
        pbar.update()
        pbar.set_postfix(train_loss=train_loss, test_loss=test_loss)

        titles = ["full_train_loss", "full_test_loss"]
        res = np.array([titles] + history)
        for j, title in enumerate(titles):
            np.savetxt(jn(path_config['reports_path'], title + '.csv'),
                        res[:, j],
                        delimiter=',',
                        fmt='%s')
        
        # checkpoint saving
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict()
            }, jn(checkp_dir, f'checkpoint{len(history)}.pt'))
        # visualizing progress
        make_visualization(jn(checkp_dir, f'visual{len(history)}'))

        os.system('dvc plots show -q')

Learning:   0%|                                                                                               …

fit_epoch:   0%|                                                                                              …

In [12]:
full_train_loss, full_test_loss = zip(*history)
res = {
    'train': {
        'loss': full_train_loss[np.argmin(full_test_loss)]
    },
    'test': {
        'loss': min(full_test_loss)
    }
}
with open(jn(path_config['reports_path'], "v_summary.json"), "w") as f:
    json.dump(res, f)

In [ ]:
!python csv_logger.py

In [22]:
if not os.path.exists(path_config['v_model_path']):
    os.makedirs(path_config['v_model_path'])
torch.save(model, jn(path_config['v_model_path'], model_name + '.pt'))


In [13]:
save_path = path_config['video_predict_vis_path']
make_visualization(save_path)

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

In [24]:
# checkpoint saving
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict()
            }, 'checkpoint.pt')

In [10]:
# history loading
train_loss = np.loadtxt(jn(path_config['reports_path'], 'full_train_loss.csv'), skiprows=1)
test_loss = np.loadtxt(jn(path_config['reports_path'], 'full_test_loss.csv'), skiprows=1)
history = [ [train, test] for train, test in zip(train_loss, test_loss)]

In [11]:
# checkpoint loading
checkpoint = torch.load(jn(checkp_dir, f'checkpoint{len(history)}.pt'))
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])

model.train()

ParamUnet(
  (step1): Sequential(
    (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (3): ReLU()
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
  )
  (finalConv): Sequential(
    (0): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (3): ReLU()
    (4): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
  )
  (bottom): Sequential(
    (0): Sequential(
      (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): ReLU()
      (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padd